In [18]:
import math
import time
import cplex
import lego
from cplex.exceptions import CplexError
import sys
from __future__ import print_function



def smartStatement(graph, prob, ub):

    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    my_upperbound = ub
    print("greedy upperbound: ", my_upperbound)
    
    for color in range(my_upperbound):
        my_obj.append(1.0) # sum y_c
        my_ub.append(1.0) #all y_c are binary
        my_colnames.append("y"+str(color))
        my_ctype += "I" #all y_c are binary
    for vertex in range(len(graph)):
        my_rhs.append(1.0) #for sum_c x_{v,c} = 1 forall v
        my_sense += "E" 
        my_rownames.append("oneColor"+str(color))
    for vertex in range(len(graph)):
        for color in range(my_upperbound):
            my_obj.append(0.0) # x_{v,c}
            my_ub.append(1.0) #all x_{v,c} are binary
            my_colnames.append("x"+str(vertex)+","+str(color))
            my_ctype += "I" #all x_{v,c} are binary
            for adjVert in range(vertex):
                my_rhs.append(1.0) #for A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
                my_sense += "L" 
                my_rownames.append("noAdjCols"+str(vertex)+","+str(color)+","+str(adjVert))
    for vertex in range(len(graph)):
        for color in range(my_upperbound):
            my_rhs.append(0.0) #for x[v,c]-y[c] <= 0 forall v
            my_sense += "L"
            my_rownames.append("colorIsUsed"+str(color))
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=my_colnames)
    rows =[]
    #sum_c x_{v,c} = 1 forall v
    for vertex in range(len(graph)):
        tmp1 = []
        tmp2 = []
        for color in range(my_upperbound):
            tmp1.append("x"+str(vertex)+","+str(color))
            tmp2.append(1.0)
        rows.append([tmp1,tmp2])
    #A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
    for vertex in range(len(graph)):
        for color in range(my_upperbound):
            for adjVert in range(vertex):
                #maybe Cplex hanles integers alright, but for now I made it long and ugly.
                rows.append([["x"+str(vertex)+","+str(color), "x"+str(adjVert)+","+str(color)],[graph[vertex][adjVert]*1.0,graph[adjVert][vertex]*1.0]])
    #x_{v,c}-y_c <= 0 forall v
    for vertex in range(len(graph)):
        for color in range(my_upperbound):
            rows.append([["y"+str(color),"x"+str(vertex)+","+str(color)],[-1.0,1.0]])
    #rownames might be removeable        
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
def statement(graph, prob):

    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    
    for color in range(len(graph)):
        my_obj.append(1.0)# sum y_c
        my_ub.append(1.0)#all y_c are binary
        my_colnames.append("y"+str(color))
        my_ctype += "I" #all y_c are binary
        my_rhs.append(1.0) #sum_c x_{v,c} = 1 forall v
        my_sense += "E" 
        my_rownames.append("oneColor"+str(color))
    for vertex in range(len(graph)):
        for color in range(len(graph)):
            my_obj.append(0.0)
            my_ub.append(1.0)
            my_colnames.append("x"+str(vertex)+","+str(color))
            my_ctype += "I" #all x_{v,c} are binary
            for adjVert in range(vertex):
                my_rhs.append(1.0) #for A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
                my_sense += "L" 
                my_rownames.append("noAdjCols"+str(vertex)+","+str(color)+","+str(adjVert))
    for vertex in range(len(graph)):
        for color in range(len(graph)):
            my_rhs.append(0.0) #for x_{v,c}-y_c <= 0 forall v
            my_sense += "L"
            my_rownames.append("colorIsUsed"+str(color))
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=my_colnames)
    rows =[]
    #sum_c(x_{v,c}) = 1 forall v
    for vertex in range(len(graph)):
        tmp1 = []
        tmp2 = []
        for color in range(len(graph)):
            tmp1.append("x"+str(vertex)+","+str(color))
            tmp2.append(1.0)
        rows.append([tmp1,tmp2])
    #A[v,u](x_{v,c}+x_{u,c}) <= 1 forall v,u
    for vertex in range(len(graph)):
        for color in range(len(graph)):
            for adjVert in range(vertex):
                #maybe Cplex hanles integers alright, but for now i made it long and ugly.
                rows.append([["x"+str(vertex)+","+str(color), "x"+str(adjVert)+","+str(color)],[graph[vertex][adjVert]*1.0,graph[adjVert][vertex]*1.0]])
    #x_{v,c}-y_c <= 0 forall v
    for vertex in range(len(graph)):
        for color in range(len(graph)):
            rows.append([["y"+str(color),"x"+str(vertex)+","+str(color)],[-1.0,1.0]])
    #rownames might be removeable        
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
    
def SchedulingForm(graph, prob, ub):
    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    my_upperbound = ub
    print("greedy upperbound: ", my_upperbound)
    
    for vertex in range(len(graph)):
        my_obj.append(0.0)
        my_ub.append(ub)
        my_colnames.append("X"+str(vertex))
        my_ctype += "I" #all colors are integral
        
        my_rhs.append(-1.0) #X_v - c =< -1 forall v
        my_sense += "L"
        my_rownames.append("less than highest color "+str(vertex))
    for vertex in range(len(graph)):
        for edge in range(vertex):
            if graph[vertex][edge] == 1:
                my_obj.append(0.0)
                my_ub.append(1.0) #x_{u,v} is binary
                my_colnames.append("x"+str(vertex)+","+str(edge))
                my_ctype += "I" #x_{u,v} is binary
                
                my_rhs.append(my_upperbound-1.0) #X_u - X_v + Kx_{u,v} =< K-1  , K >= c
                my_sense += "L" 
                my_rownames.append("1 nonAdj"+str(vertex)+","+str(edge))
                my_rhs.append(-1.0) #X_v - X_u - Kx_{u,v} =< -1
                my_sense += "L" 
                my_rownames.append("2 nonAdj"+str(vertex)+","+str(edge))
                
    my_obj.append(1.0)
    my_ub.append(ub)
    my_colnames.append("c")
    my_ctype += "I" #c is integral (but it will be even when real)
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=my_colnames)
    rows =[]
    #X_v - c =< -1 forall v
    for vertex in range(len(graph)):
        rows.append([["c","X"+str(vertex)],[-1.0,1.0]])

    for vertex in range(len(graph)):
        for edge in range(vertex):
            if graph[vertex][edge] == 1:
                #X_u - X_v + Kx_{u,v} =< K-1  , K >= c
                rows.append([["X"+str(vertex),"X"+str(edge),"x"+str(vertex)+","+str(edge)],[1.0,-1.0,my_upperbound]])
                #X_v - X_u - Kx_{u,v} =< -1
                rows.append([["X"+str(edge),"X"+str(vertex),"x"+str(vertex)+","+str(edge)],[1.0,-1.0,-my_upperbound]])    
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
    
    
def binaryStatement(graph,prob, ub):

    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    my_upperbound = ub
    binLen = int(math.ceil(math.log(ub,2)))
    print("greedy upperbound: ", my_upperbound)
    
    my_obj.append(1.0) # Max color
    my_ub.append(float(my_upperbound)) #max is our upperbound
    my_colnames.append("C")
    my_ctype += "I" #C is integral
    for vertex in range(len(graph)):
        for colorBit in range(binLen):
            my_obj.append(0.0) # x_{v,b}
            my_ub.append(1.0) #all x_{v,b} are binary
            my_colnames.append("x"+str(vertex)+","+str(colorBit))
            my_ctype += "I" #all x_{v,b} are binary
            
            for adjVert in range(vertex):
                my_obj.append(0.0) #z_{v,u,b}
                my_ub.append(1.0)
                my_colnames.append("z"+str(vertex)+","+str(adjVert)+","+str(colorBit))
                my_ctype += "I"
                my_obj.append(0.0) #t_{v,u,b}
                my_ub.append(1.0)
                my_colnames.append("t"+str(vertex)+","+str(adjVert)+","+str(colorBit))
                my_ctype += "I"
    for vertex in range(len(graph)):
        my_rhs.append(-1.0) #sum_b{2^b*x_{v,b}} - c \leq -1" forall v
        my_sense += "L"
        my_rownames.append("sum_b{2^b*x_{"+str(vertex)+",b}} - c \leq -1")
        
        for adjVert in range(vertex):
            my_rhs.append(1.0*graph[vertex][adjVert]) # sum_b{z_{v,u,b}} \geq 1*A_{v,u}
            #print("graph["+str(vertex)+"]["+str(adjVert)+"]: "+str(graph[vertex][adjVert]))
            my_sense += "G"
            my_rownames.append("sum_b{z_{"+str(vertex)+","+str(adjVert)+",b}} \geq 1")
            for colorBit in range(binLen):
                my_rhs.append(0.0) #for z_{v,u,b}- 2t_{v,u,b}+x_{v,b} -x_{u,b} = 0 forall everything
                my_sense += "E"
                my_rownames.append("|x_{"+str(vertex)+","+str(colorBit)+"} - x_{"+str(adjVert)+","+str(colorBit)+"}|")
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=my_colnames)
    rows =[]
    #sum_b{2^b*x_{v,b}} - c \geq 0" forall v
    for vertex in range(len(graph)):
        tmp1 = ["C"]
        tmp2 = [-1.0]
        for bit in range(binLen):
            tmp1.append("x"+str(vertex)+","+str(bit))
            tmp2.append(math.pow(2,bit))
        rows.append([tmp1,tmp2])
        
        # sum_b{z_{v,u,b}} \geq 1
        for adjVert in range(vertex):
            tmp1 =[]
            tmp2 =[]
            for colorBit in range(binLen):
                tmp1.append("z"+str(vertex)+","+str(adjVert)+","+str(colorBit))
                tmp2.append(1.0)
            rows.append([tmp1,tmp2])
            # z_{v,u,b}- 2t_{v,u,b}+x_{v,b} -x_{u,b} = 0 forall everything
            for colorBit in range(binLen):
                z = "z"+str(vertex)+","+str(adjVert)+","+str(colorBit)
                t = "t"+str(vertex)+","+str(adjVert)+","+str(colorBit)
                tmp1 = [z,t, "x"+str(vertex)+","+str(colorBit),"x"+str(adjVert)+","+str(colorBit) ]
                tmp2 = [1.0,-2.0,1.0,-1.0]
                rows.append([tmp1,tmp2])
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)
    
    
    
def fastBinaryStatement(graph,prob, ub):
    my_obj = []
    my_ub = []
    my_ctype = ""
    my_colnames = {}
    old_cols = []
    my_rhs = []
    my_rownames = []
    my_sense = ""
    my_upperbound = ub
    binLen = int(math.ceil(math.log(ub,2)))
    varCounter = 0
    print("greedy upperbound: ", my_upperbound)
    
    my_obj.append(1.0) # Max color
    my_ub.append(float(my_upperbound)) #max is our upperbound
    old_cols.append("C")
    my_colnames["C"] = varCounter
    varCounter += 1
    my_ctype += "I" #C is integral
    for vertex in range(len(graph)):
        for colorBit in range(binLen):
            my_obj.append(0.0) # x_{v,b}
            my_ub.append(1.0) #all x_{v,b} are binary
            old_cols.append("x{},{}".format(vertex,colorBit))
            my_colnames["x{},{}".format(vertex,colorBit)] = varCounter
            varCounter += 1
            my_ctype += "I" #all x_{v,b} are binary
            
            for adjVert in range(vertex):
                my_obj.append(0.0) #z_{v,u,b}
                my_ub.append(1.0)
                old_cols.append("z{},{},{}".format(vertex,adjVert,colorBit))
                my_colnames["z{},{},{}".format(vertex,adjVert,colorBit)]= varCounter
                varCounter += 1
                my_ctype += "I"
                my_obj.append(0.0) #t_{v,u,b}
                my_ub.append(1.0)
                old_cols.append("t{},{},{}".format(vertex,adjVert,colorBit))
                my_colnames["t{},{},{}".format(vertex,adjVert,colorBit)]= varCounter
                varCounter += 1
                my_ctype += "I"
    for vertex in range(len(graph)):
        my_rhs.append(-1.0) #sum_b{2^b*x_{v,b}} - c \leq -1" forall v
        my_sense += "L"
        my_rownames.append("sum_b{2^b*x_{"+str(vertex)+",b}} - c \leq -1")
        
        for adjVert in range(vertex):
            my_rhs.append(1.0*graph[vertex][adjVert]) # sum_b{z_{v,u,b}} \geq 1*A_{v,u}
            #print("graph["+str(vertex)+"]["+str(adjVert)+"]: "+str(graph[vertex][adjVert]))
            my_sense += "G"
            my_rownames.append("sum_b{z_{"+str(vertex)+","+str(adjVert)+",b}} \geq 1")
            for colorBit in range(binLen):
                my_rhs.append(0.0) #for z_{v,u,b}- 2t_{v,u,b}+x_{v,b} -x_{u,b} = 0 forall everything
                my_sense += "E"
                my_rownames.append("|x_{"+str(vertex)+","+str(colorBit)+"} - x_{"+str(adjVert)+","+str(colorBit)+"}|")
            
    
    prob.objective.set_sense(prob.objective.sense.minimize)
    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj=my_obj, ub=my_ub, types=my_ctype,
                       names=old_cols)
    rows =[]
    #sum_b{2^b*x_{v,b}} - c \geq 0" forall v
    for vertex in range(len(graph)):
        tmp1 = [my_colnames["C"]]
        tmp2 = [-1.0]
        for bit in range(binLen):
            tmp1.append(my_colnames["x{},{}".format(vertex,bit)])
            tmp2.append(math.pow(2,bit))
        rows.append([tmp1,tmp2])
        
        # sum_b{z_{v,u,b}} \geq 1
        for adjVert in range(vertex):
            tmp1 =[]
            tmp2 =[]
            for colorBit in range(binLen):
                tmp1.append(my_colnames["z{},{},{}".format(vertex,adjVert,colorBit)])
                tmp2.append(1.0)
            rows.append([tmp1,tmp2])
            # z_{v,u,b}- 2t_{v,u,b}+x_{v,b} -x_{u,b} = 0 forall everything
            for colorBit in range(binLen):
                z = my_colnames["z{},{},{}".format(vertex,adjVert,colorBit)]
                t = my_colnames["t{},{},{}".format(vertex,adjVert,colorBit)]
                tmp1 = [z,t,my_colnames["x{},{}".format(vertex,colorBit)], my_colnames["x{},{}".format(adjVert,colorBit)]]
                tmp2 = [1.0,-2.0,1.0,-1.0]
                rows.append([tmp1,tmp2])
    prob.linear_constraints.add(lin_expr=rows, senses=my_sense,
                                rhs=my_rhs, names=my_rownames)

def maxIndependent(graph, prob, ub):
    print("not implemented yet")
    
def cromNum(graph, method, logFileAddress):
    for row in range (1,len(graph)):
        if len(graph[row-1]) != len(graph[row]):
            print("Input a proper adjacency matrix")
            return
    start = time.time()
    try:
        my_prob = cplex.Cplex()
        my_prob.set_results_stream(logFileAddress)
        #my_prob.parameters.timelimit.set(60.0)
        my_prob.parameters.timelimit.get()
        if method == "smart":
            ub = greedyColor(graph)
            handle = smartStatement(graph, my_prob, ub)
        elif method == "Scheduling":
            ub = greedyColor(graph)
            handle = SchedulingForm(graph, my_prob, ub)
        elif method == "bin":
            ub = greedyColor(graph)
            handle = binaryStatement(graph, my_prob, ub)
        elif method == "fBin":
            ub = greedyColor(graph)
            handle = fastBinaryStatement(graph, my_prob, ub)
        else:
            ub = len(graph)
            handle = statement(graph, my_prob)
        end = time.time()
        print()
        print(method +" build problem in:")
        print(end - start)
        print()
        my_prob.solve()
    except CplexError as exc:
            print(exc)
            return

    print()
    # solution.get_status() returns an integer code
    print("Solution status = ", my_prob.solution.get_status(), ":", end= '')
    # the following line prints the corresponding string
    print(my_prob.solution.status[my_prob.solution.get_status()])
    print("Solution value  = ", my_prob.solution.get_objective_value())
    print("lower bound  = ", my_prob.solution.MIP.get_best_objective())

    
    # Print result:
    if method == "Scheduling":
        vertices = my_prob.solution.get_values(0, len(graph))
        cromnum = my_prob.solution.get_objective_value()
        for color in range(int(round(cromnum))):
            colGroup = []
            for i in range(len(graph)):
                if int(round(vertices[i])) == color:
                    colGroup.append(i + 1)
            print("vertices with colour "+str(color+1)+":", colGroup)
    elif method == "bin" or method == "fBin":
        xVals = my_prob.solution.get_values(1, ub)
        binLen = int(math.ceil(math.log(ub,2)))
        colGroups = []
    else:
        colors = my_prob.solution.get_values(0, ub)
        nextColor = 1
        for color in range(ub):
            colGroup = []
            if colors[color] > 0.5:
                for vertex in range(len(graph)):
                    if my_prob.solution.get_values("x"+str(vertex)+","+str(color)) > 0.5:
                        colGroup.append(vertex+1)
            if colGroup != []:
                print("vertices with colour "+str(nextColor)+":", colGroup)
                nextColor += 1
            
            

#    print(x)
#    for j in range(numrows):
#        print("Row %d:  Slack = %10f" % (j, slack[j]))
#    for j in range(numcols):
#        print("Column %d:  Value = %10f" % (j, x[j]))

In [10]:
def greedyColor(graph):
    vertices = len(graph)
    colors = [0]*vertices
    for vertex in range(vertices):
        neighbours = []
        for neighbour in range(vertex+1):
            if graph[vertex][neighbour] == 1:
                neighbours.append(neighbour)
        for c in range(vertices+1):
            if not any([colors[n] == c for n in neighbours]):
                break
        colors[vertex] = c
    #return max(colors)+1
    return 5

In [24]:
from itertools import product,chain,izip
from random import shuffle
# Downloaded Graphs:
# queen5_5.col
# queen6_6.col
# queen7_7.col
# queen8_8.col
# queen8_12.col
# queen9_9.col
# myciel3.col
# myciel4.col
# myciel5.col
# myciel6.col
# myciel7.col
# homer.col
def texify(filename):
    with open(filename, 'r') as original: data = original.read()
    with open(filename, 'w') as modified: modified.write("\\begin{lstlisting}[language=Python]\n" + data + '\n\\end{lstlisting}')
    
def makearr(filename):
    with open(filename, "r") as ins:
        for lines in ins:
            if lines[0] == "p":
                nodes = int(lines.split()[2])
                adjArr = [[0 for i in range(nodes)] for j in range(nodes)]
            if lines[0] == "e":
                v1 = int(lines.split()[1])-1
                v2 = int(lines.split()[2])-1
                adjArr[v1][v2] = adjArr[v2][v1] = 1
    return adjArr
def bipRandom(n):
	m = 2*n
	A = [[0 for i in range(m)] for j in range(m)]
	for i,j in product(range(0,n),range(n,m)):
		if (i*i+j*j)%17 < 4:
			A[i][j] = A[j][i] = 1
	return A
graphs ={
#'rand10': bipRandom(10),
#'rand12': bipRandom(15),
#'rand20': bipRandom(20),
#'rand25': bipRandom(25),
#'rand200': bipRandom(200),
#'myciel3': makearr('myciel3.col'),
'myciel4': makearr('myciel4.col'),
#'queen5_5':makearr('queen5_5.col'),
#'queen6_6':makearr('queen6_6.col'),
#'myciel5': makearr('myciel5.col'),
#'myciel6': makearr('myciel6.col'),
#'myciel7': makearr('myciel7.col'),
#'homer': makearr('homer.col')
}
for key, value in graphs.iteritems():
    print(key)   
    cromNum(value, "smart", "Projekt/results/{}_standard_log.tex".format(key))
    texify("Projekt/results/{}_standard_log.tex".format(key))
    cromNum(value, "fBin", "Projekt/results/{}_bin_log.tex".format(key))
    texify("Projekt/results/{}_bin_log.tex".format(key))
    cromNum(value, "Scheduling", "Projekt/results/{}_Scheduling_log.tex".format(key))
    texify("Projekt/results/{}_Scheduling_log.tex".format(key))

myciel4
greedy upperbound:  5

smart build problem in:
0.0309998989105


Solution status =  101 :MIP_optimal
Solution value  =  5.0
lower bound  =  5.0
vertices with colour 1: [3, 11, 23]
vertices with colour 2: [4, 8, 9, 15, 19, 20]
vertices with colour 3: [6, 14]
vertices with colour 4: [2, 7, 10, 13, 18, 21]
vertices with colour 5: [1, 5, 12, 16, 17, 22]
greedy upperbound:  5

fBin build problem in:
0.0230000019073


Solution status =  101 :MIP_optimal
Solution value  =  5.0
lower bound  =  5.0
greedy upperbound:  5

Scheduling build problem in:
0.0120000839233


Solution status =  101 :MIP_optimal
Solution value  =  5.0
lower bound  =  5.0
vertices with colour 1: [1, 5, 11, 23]
vertices with colour 2: [2, 4, 7, 9, 13, 15, 18, 20]
vertices with colour 3: [3, 6, 8, 12, 14, 17, 19]
vertices with colour 4: [10, 16, 21]
vertices with colour 5: [22]


In [ ]:
import time

SoerenGraph1244prime =[[0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,1],
                       [0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1],
                       [0,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,0,0,1],
                       [0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,0,0],
                       [1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,0],
                       [1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1],
                       [0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0],
                       [1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,1,0],
                       [1,0,1,0,0,1,1,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0,0,1,0,1,0,0,0,0],
                       [1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,1,1,0,0,0,0,1,0],
                       [1,1,1,0,1,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0],
                       [1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,1,0,1,1,0,0,0,0],
                       [0,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1],
                       [0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0],
                       [1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1],
                       [0,1,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1],
                       [0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,1,1,0,1,1,1,1,1,0,0,1,0],
                       [1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1],
                       [0,0,0,1,0,1,0,0,1,1,1,0,1,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0],
                       [1,0,1,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0],
                       [0,0,0,0,0,1,0,1,0,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1],
                       [0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1],
                       [0,1,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,1],
                       [0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,0,1,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0],
                       [1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,1,0,0,0,0,1,1,0,1],
                       [0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1],
                       [1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,0,0,1,0,1,0,0,0,0,1,1],
                       [0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,0],
                       [0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,1,1,1,1,0,1,0,1,1,0,1,0,0,0,0],
                       [1,0,0,1,0,0,1,1,0,0,1,0,1,1,0,0,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1],
                       [1,1,0,0,1,0,1,1,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,1,1,0,1,0,0],
                       [1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,0,1,1,1,0,0,1,0,0]]

small =[[0,0,0,0,1,1,0,1,1],
        [0,0,1,1,1,0,0,1,0],
        [0,1,0,0,0,0,1,1,1],
        [0,1,0,0,1,1,1,0,0],
        [1,1,0,1,0,0,0,0,0],
        [1,0,0,1,0,0,1,1,1],
        [0,0,1,1,0,1,0,0,1],
        [1,1,1,0,0,1,0,0,1],
        [1,0,1,0,0,1,1,1,0]]
#cromNum(SoerenGraph1244, "smart")
#cromNum(lego.legoG(3,3,10,10), "smart")
#cromNum(lego.legoG(3,3,6,6), "smart")
#cromNum(SoerenGraph3366, "smart")

cromNum(SoerenGraph1244prime,"bin")

In [ ]:
import lego
import cubes
from lego import legoG
from cubes import Qdu,Qdus 
# Generic tools for manipulating graphs

from itertools import product,chain,izip
from random import shuffle

'''
Test symmetric + zeros on diagonal
'''
def testAdj(A):
	n, m = len(A), len(A[0])
	if n!=m: raise "Not square"
	for i in range(n): 
		if A[i][i]==1: raise "Loops"
	for i,j in product(range(n), range(n)):
		if A[i][j]!=A[j][i]: raise "Not symmetric"
	return A

'''
Canonicalize the graph into an adjacency matrix using the given vertex set
'''
def canonical(vertices, edges):
	n = len(vertices)
	v = { x:i for x,i in izip(vertices, range(n)) }
	A = [[0 for i in range(n)] for j in range(n)]
	for e in edges:
		A[v[e[0]]][v[e[1]]] = 1
	return testAdj(A)

'''
Print graph (adj matrix) to a file in DIMACS compatible format
'''
def printDimacs(A, filename):
	n = len(A)
	m = sum([sum(x) for x in A]) // 2
	with open(filename, 'w') as f:
		f.write("p edge {0} {1}\n".format(n, m))
		for i in range(n):
			for j in range(i,n):
				if A[i][j]==1:
					f.write("e {0} {1}\n".format(i+1, j+1))

'''
Read graph in DIMACS format. Return adjacency matrix.
Vertices are numbered 1..n, we map it to 0..n-1
'''
def readDimacs(filename):
	v = 0
	e = 0
	edgesBoth = []
	with open(filename) as f:
		for line in f.readlines():
			if line[0]=="p":
				_, _, v, e = line.split()
				v, e = int(v), int(e)
			if line[0]=="e":
				_, x, y = line.split()
				edgesBoth.append((int(x)-1,int(y)-1))				
				edgesBoth.append((int(y)-1,int(x)-1))
	return canonical(range(v), edgesBoth)

'''
Return graph complement in matrix format
'''
def complement(A):
	n = len(A)
	B = [[1-x for x in row] for row in A]
	for i in range(n): B[i][i] = 0
	return B

'''
Return the edge list of a graph from matrix format
'''
def intoEdges(A):
	n = len(A)
	return [ (x,y) for x,y in product(range(n),range(n)) if A[x][y]==1 ]

'''
Return the best greedy chromatic number of A over a course of s random vertex orderings
'''
def bestGreedyChi(A, s):
	n = len(A)
	best = 10**9
	for _ in range(s):
		o = range(n)
		shuffle(o)
		col = [-1] * n
		for x in o:
			used = [False] * n
			for y in range(n):
				if A[x][y]==1 and col[y]!=-1:
					used[col[y]] = True
			col[x] = min([c for c in range(n) if not used[c]])
		best = min(best, max(col)+1)
	return best

'''
Random bipartite graph on n vertices
'''
def bipRandom(n):
	m = 2*n
	A = [[0 for i in range(m)] for j in range(m)]
	for i,j in product(range(0,n),range(n,m)):
		if (i*i+j*j)%17 < 4:
			A[i][j] = A[j][i] = 1
	return A
ub = 0
timeLimit = 0
testCases = {
    # Queen graphs
    "queen5_5": (readDimacs('tests/data/queen5_5.col'), ub, timeLimit),
    "queen6_6": (readDimacs('tests/data/queen6_6.col'), ub, timeLimit),
    "queen7_7": (readDimacs('tests/data/queen7_7.col'), ub, timeLimit),
    "queen8_8": (readDimacs('tests/data/queen8_8.col'), ub, timeLimit),
    "queen9_9": (readDimacs('tests/data/queen9_9.col'), ub, timeLimit),
    "queen10_10": (readDimacs('tests/data/queen10_10.col'), ub, timeLimit),
    "queen8_12": (readDimacs('tests/data/queen8_12.col'), ub, timeLimit),
    # Mycielski graphs
    "myciel3": (readDimacs('tests/data/myciel3.col'), 4, timeLimit),
    "myciel4": (readDimacs('tests/data/myciel4.col'), 5, timeLimit),
    "myciel5": (readDimacs('tests/data/myciel5.col'), 6, timeLimit),
    "myciel6": (readDimacs('tests/data/myciel6.col'), 7, timeLimit),
    "myciel7": (readDimacs('tests/data/myciel7.col'), 8, timeLimit),
    # Real application (register allocation) graphs (large chromatic numbers!)
    "reg1": (readDimacs('tests/data/zeroin.i.1.col'), 50, timeLimit),
    "reg2": (readDimacs('tests/data/zeroin.i.2.col'), 50, timeLimit),
    "reg3": (readDimacs('tests/data/zeroin.i.3.col'), 50, timeLimit),
    "reg4": (readDimacs('tests/data/mulsol.i.1.col'), 50, timeLimit),
    "reg5": (readDimacs('tests/data/mulsol.i.2.col'), 50, timeLimit),
    # (Almost) real application - proximity graphs of US road network at various scales, V=128 always
    "miles250": (readDimacs('tests/data/miles250.col'), 10, timeLimit),
    "miles500": (readDimacs('tests/data/miles500.col'), 30, timeLimit),
    "miles750": (readDimacs('tests/data/miles750.col'), 35, timeLimit),
    "miles1000": (readDimacs('tests/data/miles1000.col'), 48, timeLimit),
    "miles1500": (readDimacs('tests/data/miles1500.col'), 100, timeLimit),
    # Some of the LEGO graphs
    # G(2,2)
    "G_2_2_6_6": (legoG(2,2,6,6), 5, timeLimit),
    "G_2_2_8_8": (legoG(2,2,8,8), 8, timeLimit),    
    "G_2_2_9_10": (legoG(2,2,9,10), 8, timeLimit),
    "G_2_2_10_10": (legoG(2,2,10,10), 5, timeLimit),    
    # G(3,3)
    "G_3_3_6_6": (legoG(3,3,6,6), 7, timeLimit),
    "G_3_3_8_8": (legoG(3,3,8,8), 7, timeLimit),
    "G_3_3_10_10": (legoG(3,3,10,10), 7, timeLimit),    
    "G_3_3_12_12": (legoG(3,3,12,12), 7, timeLimit),    
    # G(1,2)                                    
    # For G(1,2) it is also interesting to ask directly if they are 4-colorable ! May be faster to decide    
    # And also to compare when upper bound is smaller
    "G_1_2_4_4": (legoG(1,2,4,4), 8, timeLimit),
    "G_1_2_4_6": (legoG(1,2,4,6), 8, timeLimit),
    "G_1_2_4_10": (legoG(1,2,4,10), 8, timeLimit),
    "G_1_2_4_12": (legoG(1,2,4,12), 8, timeLimit),
    "G_1_2_6_6": (legoG(1,2,6,6), 8, timeLimit),
    "G_1_2_6_10": (legoG(1,2,6,10), 8, timeLimit),
    "G_1_2_6_12": (legoG(1,2,6,12), 8, timeLimit),
    "G_1_2_8_12": (legoG(1,2,8,10), 8, timeLimit),    
    "G_1_2_10_12": (legoG(1,2,10,12), 8, timeLimit),        
    "G_1_2_12_12": (legoG(1,2,12,12), 8, timeLimit),  
    # Generalized cube graphs
    "Q_7_4": (Qdu(7,4), 10, timeLimit),          
    "Q_8_2": (Qdu(8,2), 10, timeLimit),          
    "Q_8_4": (Qdu(8,4), 10, timeLimit),          
    "Q_9_2": (Qdu(9,2), 20, timeLimit),          
    "Q_9_4": (Qdu(9,4), 22, timeLimit),          
    "Q_10_4_3": (Qdus(10,4,3), 28, timeLimit),                      # Increase 28 if necessary - I'm not sure what the aswer is!. See when it finds something feasible
    "Q_10_4_5": (Qdus(10,4,5), 28, timeLimit),
    # Random bipartite graph
    "bip50": (bipRandom(50), 3, timeLimit),
    "bip200": (bipRandom(200), 3, timeLimit),
    "bip500": (bipRandom(500), 3, timeLimit),
}

# Get only tests whose name matches a substring
def getByName(cases, str):
    return sorted([ name for name in testCases if str in name ])

# Go through all tests (or all tests in a group) and run them
# AHA: It would maybe be cool to also get the running time in case when it finished early i.e. before time limit
allTests = getByName(testCases, "")

for name in allTests:
    case = testCases[name]
    graph = case[0]
    vert = len(graph)
    upper = case[1]
    print(name, bestGreedyChi(graph, 30))


In [10]:
#read txt
with open("all.txt", "r") as ins:
#TEX
    with open('Projekt/results/results.tex', 'w') as f:
        #f.write('\\begin{table}[]\n')
        f.write('\\centering\n')
        f.write('\\begin{longtable}{|lll|lll|lll|lll|}\n')
        f.write('\\hline\n\\multicolumn{3}{|c|}{Graph}&\\multicolumn{3}{c}{Standard}&\\multicolumn{3}{c}{Scheduling} &\\multicolumn{3}{c}{Binary}\\\\')
        f.write('\nName&$|V|$&$k$&lb&ub&time&lb&ub&time&lb&ub&time\\\\')
        f.write('\n\\hline')
        for lines in ins:
            case = lines.split()
            name = case[0]
            vert = case[1]
            greedy = case[2]
            lbStand = case[3]
            ubStand = case[4]
            timeStand = case[5]
            lbSche = case[6]
            ubSche = case[7]
            timeSche = case[8]
            lbBin = case[9]
            ubBin = case[10]
            timeBin = case[11]
            #write graph name #write standard
            f.write('\n{0}&{1}&{2}&{3}&{4}&{5}&{6}&{7}&{8}&{9}&{10}&{11}\\\\\n\\hline'.format(name.replace('_', '\_'),vert, greedy,lbStand,ubStand, timeStand,lbSche,ubSche, timeSche,lbBin,ubBin, timeBin))
        f.write('\n\\caption{Results}')
        f.write('\n\\label{table}')
        f.write('\n\\end{longtable}')